# Preprocessor Tuning

In [38]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import set_config; set_config(display='diagram')

## (0) The turmors Dataset

* The following dataset describes tumors that are either <font color=red>malignant</font> or <font color=green>benign</font>. 
* The task is to detect as many malignant tumors as possible.

In [3]:
data = pd.read_csv("../data/workflow/tumors_dataset.csv")
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,malignant
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,NaN,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,NaN,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.2,1575.0,NaN,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [4]:
data.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,malignant
count,568.000000,569.000000,567.000000,568.000000,568.000000,569.000000,567.000000,567.000000,569.000000,569.000000,...,566.000000,566.000000,567.000000,564.000000,564.000000,565.000000,566.000000,566.000000,567.000000,569.000000
mean,14.125650,19.289649,91.884568,655.126937,0.096319,0.104341,0.088756,0.048845,0.181162,0.062798,...,25.662792,107.370583,881.842504,0.132332,0.254284,0.271640,0.114679,0.290069,0.083956,0.372583
std,3.526937,4.301036,24.273297,352.178550,0.014042,0.052813,0.079754,0.038846,0.027414,0.007060,...,6.158413,33.651145,569.965640,0.022906,0.157231,0.208576,0.065735,0.061893,0.018091,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.697500,16.170000,75.100000,420.175000,0.086290,0.064920,0.029520,0.020310,0.161900,0.057700,...,21.065000,84.122500,514.650000,0.116500,0.147575,0.114400,0.064943,0.250425,0.071365,0.000000
50%,13.355000,18.840000,86.180000,551.400000,0.095865,0.092630,0.061540,0.033410,0.179200,0.061540,...,25.370000,97.665000,686.600000,0.131300,0.213000,0.226700,0.100015,0.282250,0.080040,0.000000
75%,15.797500,21.800000,103.950000,784.150000,0.105300,0.130400,0.130000,0.073820,0.195700,0.066120,...,29.705000,125.775000,1086.000000,0.146025,0.339300,0.382900,0.161375,0.317825,0.092085,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,1.000000


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              568 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           567 non-null    float64
 3   mean area                568 non-null    float64
 4   mean smoothness          568 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           567 non-null    float64
 7   mean concave points      567 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            567 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [9]:
print(data.malignant.value_counts(normalize=True))
data.malignant.value_counts()

malignant
0    0.627417
1    0.372583
Name: proportion, dtype: float64


malignant
0    357
1    212
Name: count, dtype: int64

## (1) Building a Pipeline

In [18]:
pipeline = Pipeline([
    ('imputer', KNNImputer()),
    ('scaling', MinMaxScaler()),
    ('model', LogisticRegression())
])

pipeline

,steps,"[('imputer', ...), ('scaling', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,n_neighbors,5
,weights,'uniform'
,metric,'nan_euclidean'
,copy,True
,add_indicator,False
,keep_empty_features,False


## (2) Optmizing a pipelined model

In [ ]:
X = data.drop(columns=['malignant'])
y = data.malignant
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

grid_search = GridSearchCV(
    pipeline,
    param_grid={
        'imputer__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    },
    cv=10,
    scoring='recall'
)
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'imputer__n_neighbors': 2}

In [29]:
grid_search = GridSearchCV(
    pipeline,
    param_grid={
        'imputer__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    },
    cv=5,
    scoring='recall'
)
grid_search.fit(data.drop(columns=['malignant']), data.malignant)
grid_search.best_params_

{'imputer__n_neighbors': 3}

In [30]:
n_best = grid_search.best_params_["imputer__n_neighbors"]
n_best

3

## (3) Evaluating a pipeline

In [39]:
X = data.drop(columns=['malignant'])
y = data.malignant
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

cv_score = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=5, scoring='recall').mean()
cv_score

np.float64(0.9056451612903226)

## (4) Predicting using a fitted and pipelined model

In [41]:
new_tumor = pd.read_csv("../data/workflow/new_tumor.csv")
new_tumor.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.186,0.275,0.08902


In [ ]:
X = data.drop(columns=['malignant'])
y = data.malignant

estimator = grid_search.best_estimator_

# Make a prediction with pipeline (best estimator)
estimator.predict(new_tumor)

array([1])

In [43]:
# FYI, you have access to the predicted probabilities
grid_search.best_estimator_.predict_proba(new_tumor)

array([[0.03194086, 0.96805914]])